In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import pandas as pd
import numpy as np

# lgb xgb 앙상블


In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [0]:
train = train.loc[:, train.max() != train.min()]
test = test.loc[:, test.max() != test.min()]

In [0]:
train['Y18'].isnull().sum(), train['Y18'].notnull().sum()
train_first_30 = train[train['Y18'].isnull()]
train_first_30.shape
train_later_3 = train[train['Y18'].notnull()]
train_later_3.shape

In [0]:

X_train = train_first_30.loc[:, 'X00':'X39']
X_train
X_train = pd.concat([X_train, X_train], axis=0)
X_train = pd.concat([X_train, X_train], axis=0)
X_train = pd.concat([X_train, train_later_3.loc[:, 'X00':'X39']], axis=0)
X_train = pd.concat([X_train, train_later_3.loc[:, 'X00':'X39']], axis=0)
X_train = pd.concat([X_train, train_later_3.loc[:, 'X00':'X39']], axis=0)
X_train

In [0]:
def momentValue(df, columns):
    for column in columns:
        temp_df = pd.DataFrame(df[column])
        temp_df['dummy'] = 0
        
        for i in range(1, temp_df.shape[0]):
            a = temp_df[column].iloc[i] - temp_df[column].iloc[i-1]
            if a > 0:
                temp_df['dummy'].iloc[i] = a
        
        df[column] = temp_df['dummy']
    
    return df

In [0]:
X_train = momentValue(X_train,['X11','X34'])
X_train = momentValue(X_train,['X04','X10','X21','X36','X39'])
test = momentValue(test,['X11','X34'])
test = momentValue(test,['X04','X10','X21','X36','X39'])

In [0]:
def scale(df):
  df/=df.max()
  return df



In [0]:
select_Tr = X_train.loc[:,['X13','X15','X17','X25','X35','X01','X06','X22','X27','X29','X05','X08','X09','X23','X33']]
select_Ts = test.loc[:,['X13','X15','X17','X25','X35','X01','X06','X22','X27','X29','X05','X08','X09','X23','X33']]

In [0]:
stand_Tr = X_train.loc[:,['X00','X02','X03','X04','X07','X10','X11','X12','X18','X20','X21','X24','X26','X28','X30','X31','X32','X34','X37','X38','X39']]
stand_Ts = test.loc[:,['X00','X02','X03','X04','X07','X10','X11','X12','X18','X20','X21','X24','X26','X28','X30','X31','X32','X34','X37','X38','X39']]

In [0]:
scale(select_Tr)
scale(select_Ts)

In [0]:
#표준화 함수 생성
def standardization(df):
    mean = np.mean(df)
    std = np.std(df)
    norm = (df - mean) / (std - 1e-07)
    return norm, mean, std

In [0]:
# 학습데이터 표준화
X_train_norm, mean, std = standardization(stand_Tr)

In [0]:
y_train = train_first_30.loc[:, 'Y16']
y_train = pd.concat([y_train, y_train], axis=0)
y_train = pd.concat([y_train, y_train], axis=0)
y_train = pd.concat([y_train, train_later_3.loc[:, 'Y18']], axis=0)
y_train = pd.concat([y_train, train_later_3.loc[:, 'Y18']], axis=0)
y_train = pd.concat([y_train, train_later_3.loc[:, 'Y18']], axis=0)
y_train

In [0]:
X_test_norm = (stand_Ts - mean) / (std - 1e-07)

In [0]:
X_train_norm = pd.concat([X_train_norm,select_Tr],axis=1)
X_test_norm = pd.concat([X_test_norm,select_Ts],axis=1)

In [0]:
# ID로 시간 변수 생성
minute = pd.Series((X_train_norm.index%144).astype(int))
hour= pd.Series((X_train_norm.index%144/6).astype(int))

# 삼각함수를 이용한 시간변수 생성
min_in_day = 24*6
hour_in_day = 24

minute_sin = np.sin(np.pi*minute/min_in_day) 
minute_cos = np.cos(np.pi*minute/min_in_day)

hour_sin  = np.sin(np.pi*hour/hour_in_day)
hour_cos  = np.cos(np.pi*hour/hour_in_day)

X_train_norm['minute_sin'] = minute_sin
X_train_norm['minute_cos'] = minute_cos

X_train_norm['hour_sin'] = hour_sin
X_train_norm['hour_cos'] = hour_cos
X_train_norm

In [0]:

# ID로 시간 변수 생성
minute = pd.Series((test.id%144).astype(int))
hour= pd.Series((test.id%144/6).astype(int))

# 삼각함수를 이용한 시간변수 생성
min_in_day = 24*6
hour_in_day = 24

minute_sin = np.sin(np.pi*minute/min_in_day) 
minute_cos = np.cos(np.pi*minute/min_in_day)

hour_sin  = np.sin(np.pi*hour/hour_in_day)
hour_cos  = np.cos(np.pi*hour/hour_in_day)

In [0]:
X_test_norm['minute_sin'] = minute_sin
X_test_norm['minute_cos'] = minute_cos

X_test_norm['hour_sin'] = hour_sin
X_test_norm['hour_cos'] = hour_cos
X_test_norm

In [0]:
X_train_norm = X_train_norm.loc[:, X_train_norm.max() != X_train_norm.min()]
X_test_norm = X_test_norm.loc[:, X_test_norm.max() != X_test_norm.min()]
len(X_test_norm.columns) == len(X_train_norm.columns)

In [0]:

# 실제값과 예측값의 차이가 1보다 작은 건 무시하는 Custom MSE Metric
def mse1(y_pred, y_true):

    diff = abs(y_true - y_pred)
    mask_less_than_one = np.array([0 if x < 1 else 1 for x in diff])

    y_pred = mask_less_than_one * y_pred
    y_true = mask_less_than_one *y_true
    
    score =  mean_squared_error(y_true, y_pred)
    
    return 'score', score, False

In [0]:
X_train_30 = X_train_norm.iloc[:4320+4320+4320,:]
y_train_30 = y_train.iloc[:4320+4320+4320]
X_train_3 = X_train_norm.iloc[4320+4320+4320:,:]
y_train_3 = y_train.iloc[4320+4320+4320:]

In [0]:
selected_columns = ['X30',
                    'X00',
                    'X38',
                    'X20',
                    'X32',
                    'X37',
                    'X34',
                    'X07',
                    'X12','X13','X31','X11','X28','X18','minute_sin','X26','X02','X03','minute_cos','X01'
                    ]
                    #minute_sin,X37,minute_cos,X07,X38,X01,X20,X31,X11,X00,X29,hour_cos

In [0]:
X_train_30 =X_train_30[selected_columns]

X_train_3 = X_train_3[selected_columns]
test = X_test_norm[selected_columns]

In [0]:
pip install shap

In [0]:
import pandas as pd
import numpy as np

from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import gc

import shap

In [0]:
DEBUG = False

if DEBUG:
    NFOLDS = 2
    N_ESTIMATORS = 50
else:
    NFOLDS = 5
    N_ESTIMATORS = 2500

In [0]:

# Y15, Y16, Y18을 활용한 훈련데이터 세트 모델링

feature_importance = pd.DataFrame()
feature_importance['feature'] = X_train_30.columns

folds = KFold(n_splits=NFOLDS, shuffle=True)
y_preds = np.zeros(test.shape[0])

model = LGBMRegressor(n_estimators=N_ESTIMATORS,
                       boosting_type='gbdt',
                       num_leaves=60,
                       max_depth=-1,
                       learning_rate=0.01,
                       objective='regression',
                       sub_sample=0.7,
                       colsample_bytree=0.8,
                       min_data_in_leaf=100,
                       bagging_seed=11,
                      metrics="mae",
                       # metric=mse_ignoring_diff_less_than_one,
                       verbosity=-1,
                       random_state=47)

for fold_n, (train_index, valid_index) in enumerate(folds.split(X_train_30, y_train_30)):
    print('Fold: ', fold_n+1)
    X_train_fold, X_valid_fold = X_train_30.iloc[train_index], X_train_30.iloc[valid_index]
    y_train_fold, y_valid_fold = y_train_30.iloc[train_index], y_train_30.iloc[valid_index]
    
    evals = [(X_train_fold, y_train_fold), (X_valid_fold,  y_valid_fold)]
    model.fit(X_train_fold, 
              y_train_fold, 
              eval_metric=mse1,
              eval_set=evals,
              early_stopping_rounds=50,
              verbose=100
             )
    
    shap_values = shap.TreeExplainer(model.booster_).shap_values(X_train_fold)
    fold_importance = pd.DataFrame()
    fold_importance['feature'] = X_train_30.columns
    fold_importance['shap_values'] = abs(np.array(shap_values)[:, :].mean(0))
    fold_importance['feat_importance'] = model.feature_importances_
    feature_importance = pd.concat([feature_importance, fold_importance], sort=False)
        
    y_preds += model.predict(test) / NFOLDS
    
    del X_train_fold, X_valid_fold, y_train_fold, y_valid_fold
    gc.collect()

feature_importance = feature_importance.groupby('feature').mean().sort_values('shap_values', ascending=False).reset_index()

In [0]:

# Y15, Y16, Y18을 활용한 훈련데이터 세트 모델링

feature_importance = pd.DataFrame()
feature_importance['feature'] = X_train_3.columns

folds = KFold(n_splits=NFOLDS, shuffle=True)
y_preds_3 = np.zeros(test.shape[0])

model_3 = LGBMRegressor(n_estimators=N_ESTIMATORS,
                       boosting_type='gbdt',
                       num_leaves=60,
                       max_depth=-1,
                       learning_rate=0.01,
                       objective='regression',
                      #metrics="mae",
                       sub_sample=0.7,
                       colsample_bytree=0.8,
                       min_data_in_leaf=100,
                       bagging_seed=11,
                       # metric=mse_ignoring_diff_less_than_one,
                       verbosity=-1,
                       random_state=47)

for fold_n, (train_index, valid_index) in enumerate(folds.split(X_train_3, y_train_3)):
    print('Fold: ', fold_n+1)
    X_train_fold, X_valid_fold = X_train_3.iloc[train_index], X_train_3.iloc[valid_index]
    y_train_fold, y_valid_fold = y_train_3.iloc[train_index], y_train_3.iloc[valid_index]
    
    evals = [(X_train_fold, y_train_fold), (X_valid_fold,  y_valid_fold)]
    model_3.fit(X_train_fold, 
              y_train_fold, 
              eval_metric=mse1,
              eval_set=evals,
              early_stopping_rounds=50,
              verbose=100
             )
    
    shap_values = shap.TreeExplainer(model_3.booster_).shap_values(X_train_fold)
    fold_importance = pd.DataFrame()
    fold_importance['feature'] = X_train_3.columns
    fold_importance['shap_values'] = abs(np.array(shap_values)[:, :].mean(0))
    fold_importance['feat_importance'] = model_3.feature_importances_
    feature_importance = pd.concat([feature_importance, fold_importance], sort=False)
        
    y_preds_3 += model_3.predict(test) / NFOLDS
    
    del X_train_fold, X_valid_fold, y_train_fold, y_valid_fold
    gc.collect()

feature_importance = feature_importance.groupby('feature').mean().sort_values('shap_values', ascending=False).reset_index()

In [0]:

ensemble_lgb = 0.6 * y_preds + 0.4 * y_preds_3



In [0]:
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_train_30,y_train_30,test_size=0.4,shuffle=True)

dtrain=xgb.DMatrix(X_train, label=y_train)
del(X_train)
dtest = xgb.DMatrix(X_test)
del(X_test)
def xgb_evaluate(max_depth, gamma, colsample_bytree,scale_pos_weight,learning_rate):
    params = {'eval_metric': 'mae',
              'max_depth': int(max_depth),
              'subsample': 0.8,
              'eta': 0.1,
              'learning_rate':learning_rate,
              'nthread':10,
              'gamma': gamma,
              'scale_pos_weight':scale_pos_weight,
              'colsample_bytree': colsample_bytree,
              'seed':7}
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round=100, nfold=5)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -1.0 * cv_result['test-mae-mean'].iloc[-1]
xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 7), 
                                             'gamma': (0, 1),
                                             'colsample_bytree': (0.3, 0.9),
                                             'scale_pos_weight':(1,10),
                                             'learning_rate':(0.0001,0.2)})
# Use the expected improvement acquisition function to handle negative numbers
# Optimally needs quite a few more initiation points and number of iterations
xgb_bo.maximize(init_points=5, n_iter=50, acq='ei')

In [0]:
targets = []
for i, rs in enumerate(xgb_bo.res):
    targets.append(rs["target"])
best_params = xgb_bo.res[targets.index(max(targets))]["params"]
best_params["max_depth"] = int(best_params["max_depth"])

In [0]:
model2 = xgb.train(best_params, dtrain, num_boost_round=2500,feval=mse1)

# Predict on testing and training set
y_pred = model2.predict(dtest)
y_train_pred = model2.predict(dtrain)

# Report testing and training RMSE
print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(np.sqrt(mean_squared_error(y_train, y_train_pred)))

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_train_3,y_train_3,test_size=0.4,shuffle=True)

dtrain=xgb.DMatrix(X_train, label=y_train)
del(X_train)
dtest = xgb.DMatrix(X_test)
del(X_test)
def xgb_evaluate(max_depth, gamma, colsample_bytree,scale_pos_weight,learning_rate):
    params = {'eval_metric': 'mae',
              'max_depth': int(max_depth),
              'subsample': 0.8,
              'eta': 0.1,
              'learning_rate':learning_rate,
              'nthread':10,
              'gamma': gamma,
              'scale_pos_weight':scale_pos_weight,
              'colsample_bytree': colsample_bytree,
              'seed':7}
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round=100, nfold=5)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -1.0 * cv_result['test-mae-mean'].iloc[-1]
xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 7), 
                                             'gamma': (0, 1),
                                             'colsample_bytree': (0.3, 0.9),
                                             'scale_pos_weight':(1,10),
                                             'learning_rate':(0.0001,0.2)
                                             })
# Use the expected improvement acquisition function to handle negative numbers
# Optimally needs quite a few more initiation points and number of iterations
xgb_bo.maximize(init_points=5, n_iter=50, acq='ei')

In [0]:
targets = []
for i, rs in enumerate(xgb_bo.res):
    targets.append(rs["target"])
best_params = xgb_bo.res[targets.index(max(targets))]["params"]
best_params["max_depth"] = int(best_params["max_depth"])

In [0]:
model3 = xgb.train(best_params, dtrain, num_boost_round=2500,feval=mse1)

# Predict on testing and training set
y_pred = model3.predict(dtest)
y_train_pred = model3.predict(dtrain)

# Report testing and training RMSE
print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(np.sqrt(mean_squared_error(y_train, y_train_pred)))

In [0]:
X_test_norm = xgb.DMatrix(test)
predict_30 = model2.predict(X_test_norm)
predict_3 = model3.predict(X_test_norm)

In [0]:
ensembled_xgb = 0.6 * predict_30 + 0.4 * predict_3

In [0]:
ensembled_lgxg = ensembled_xgb*0.4 + ensemble_lgb*0.6
submit = pd.DataFrame({'id':range(144*33, 144*113),
              'Y18':ensembled_lgxg})
submit.to_csv('final.csv', index = False)